In [1]:
import ipywidgets as widgets 
from IPython.display import display, clear_output
import pyperclip

!python -m spacy download en_core_web_md

#inputs
prompt_box = widgets.Text(
    value="Are NFT's expensive?",
    placeholder="Type your question or instruction",
    description='prompt:',
    disabled=False,
    layout=widgets.Layout(width="90%")
)

role_box = widgets.Text(
    value="A person building reputation as NFT expert by posting Tweets on Tweeter",
    placeholder="Type the role you want the model to play",
    description='role:',
    disabled=False,
    layout=widgets.Layout(width="90%")   
)

openAIAPIKey = widgets.Text(
    value="",
    placeholder="",
    description='openAI key:',
    disabled=False,
    layout=widgets.Layout(width="90%")   
)

searchAPIKey = widgets.Text(
    value="",
    placeholder="",
    description='search key:',
    disabled=False,
    layout=widgets.Layout(width="90%")   
)

selfImprovementLoop_checkbox = widgets.Checkbox(
    value=False,
    description='Self-improv',
    disabled=False,
    indent=False
)

selfImprovementLoopCount_int = widgets.IntSlider(value=2, min=0,max=10,step=1, description='# slf-imp loops :')

slider_sourceNum = widgets.IntSlider(value=4, min=0,max=10,step=1, description='# pages :')
slider_Temperature = widgets.FloatSlider(
    value=0.6,
    min=0,
    max=1,
    step=0.1,
    description='Rand :',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

model_dropdown = widgets.Dropdown(
    options=["gpt-3.5-turbo","gpt-4"],
    value="gpt-3.5-turbo",
    description='model:',
    disabled=False,
)


namedEntitiesDisplay = widgets.Checkbox(
    value=False,
    description='Named Entities Display',
    disabled=False,
    indent=False
)

extractBool = widgets.Checkbox(
    value=False,
    description='Use extract from last output',
    disabled=False,
    indent=False
)

extractedText = widgets.Textarea(
    value="",
    description='last extract',
    disabled=True,
    indent=False
)

def on_text_paste_openAi(_):
    openAIAPIKey.value = pyperclip.paste()

paste_button_openAI = widgets.Button(description='Paste')
paste_button_openAI.on_click(on_text_paste_openAi)



def on_text_paste_search(_):
    searchAPIKey.value = pyperclip.paste()

paste_button_search = widgets.Button(description='Paste')
paste_button_search.on_click(on_text_paste_search)

openAIEntry = widgets.HBox([openAIAPIKey,paste_button_openAI])
searchEntry = widgets.HBox([searchAPIKey,paste_button_search])


inputs = widgets.VBox([openAIEntry,searchEntry,prompt_box,role_box],layout=widgets.Layout(height="100%", width="50%"))
parameters = widgets.VBox([slider_sourceNum,slider_Temperature,selfImprovementLoop_checkbox,selfImprovementLoopCount_int,model_dropdown])
efficiency = widgets.VBox([extractedText,extractBool,namedEntitiesDisplay])
input_pannel = widgets.HBox([inputs,parameters,efficiency])



#outputs
output = widgets.Output()

improvement_txt = None
twt_txt = None

# set up GUI
btn_regen = widgets.Button(description='Generate New')

def updateWidgetsWithOutputDisplayed(tweet, namedEntities, linksList,extractedTextIn,improvLoopTraceList):

        extractedText.value = extractedTextIn
        
        global twt_txt

        twt_txt = widgets.Textarea(
            value=tweet,
            placeholder='Type something',
            description='Tweet:',
            disabled=True,
            layout=widgets.Layout(height="100%", width="auto")
        )
        entities_txt = widgets.Textarea(
            value=str(namedEntities),
            placeholder='',
            description='Named entities detected:',
            disabled=True,
            layout=widgets.Layout(height="100%", width="auto")
        )
        links_txt = widgets.Textarea(
            value=str(linksList),
            placeholder='Type something',
            description='Used Links:',
            disabled=True,
            layout=widgets.Layout(height="100%", width="auto")
        )

        extract_txt = widgets.Textarea(
            value=extractedText.value ,
            placeholder='Extracted Context',
            description='Extracted Context: ',
            disabled=True,
            layout=widgets.Layout(height="100%", width="auto")
        )

        selfImprovementLoopTraceback = widgets.Textarea(
            value=str(improvLoopTraceList),
            placeholder='',
            description='Self Improvement loop trace: ',
            disabled=True,
            layout=widgets.Layout(height="100%", width="auto")
        )

        global improvement_txt
        improvement_txt = widgets.Textarea(
            value="",
            placeholder='Incorporate more humour into the tweet',
            description='Improvement: ',
            disabled=False,
            layout=widgets.Layout(width="50%")
        )


        btn_improve = widgets.Button(description='Improve')
        btn_improve.on_click(improveOutput)

        improvement_widget = widgets.HBox([improvement_txt,btn_improve])

        display(widgets.VBox([twt_txt,entities_txt,links_txt,extract_txt,selfImprovementLoopTraceback,improvement_widget]))

def generate_new_output(_):
    from tweet_generator import simplePromptTweetGeneration
    with output:
        clear_output()

        if(extractBool.value):
            preExtractedTextIn = extractedText.value
        else: 
            preExtractedTextIn = ""

        tweet, namedEntities, linksList,extrText,improvLoopTraceList = simplePromptTweetGeneration(prompt_box.value,slider_sourceNum.value, role_box.value,selfImprovementLoop_checkbox.value,slider_Temperature.value, namedEntitiesDisplayValue= namedEntitiesDisplay.value,preExtractedText = preExtractedTextIn,modelNameIn=model_dropdown.value,selfImprovementLimitCount = selfImprovementLoopCount_int.value, apiKeyOpenAI= openAIAPIKey.value,searchAPIKey=searchAPIKey.value)
        updateWidgetsWithOutputDisplayed(tweet, namedEntities, linksList,extrText,improvLoopTraceList)
        
def improveOutput(_):
    from tweet_generator import simplePromptTweetGeneration
    with output:

        preExtractedTextIn = extractedText.value

        tweet, namedEntities, linksList,extrText,improvLoopTraceList = simplePromptTweetGeneration(prompt_box.value,slider_sourceNum.value, role_box.value,False,slider_Temperature.value, namedEntitiesDisplayValue= namedEntitiesDisplay.value,preExtractedText = preExtractedTextIn,modelNameIn=model_dropdown.value,requestType="imp",currTweet=twt_txt.value,selfImprovementLimitCount = selfImprovementLoopCount_int.value,apiKeyOpenAI= openAIAPIKey.value,searchAPIKey=searchAPIKey.value)
        updateWidgetsWithOutputDisplayed(tweet, namedEntities, linksList,extrText,improvLoopTraceList)

btn_regen.on_click(generate_new_output)


box = widgets.VBox([input_pannel,btn_regen,output])

display(box)
